In [1]:
import sys
sys.path.append("..")

In [2]:
%load_ext autoreload
%autoreload 2

In [38]:
from src.dataset import HierarchicalDataset
import pandas as pd

In [46]:
dataset = HierarchicalDataset(config_dir="../../config/catalog.yml", 
                              cases_dir="../../data/COVID-19-up-to-date.csv", 
                              ifr_dir="../../data/popt_ifr.csv", 
                              serial_interval_dir="../../data/serial_interval.csv", 
                              interventions_dir="../../data/interventions.csv", num_covariates=6)

In [47]:
dataset.ifr

,country,popt,ifr
1,Denmark,5792203,0.010207
2,Norway,5421242,0.009150
3,Sweden,10099270,0.010311
4,United_Kingdom,67886004,0.010350
5,Italy,60461828,0.012450
6,Spain,46754783,0.010784
7,Austria,9006400,0.010388
8,Belgium,11589616,0.010960
9,France,65273512,0.012556
10,Germany,83783945,0.012332


In [65]:
stan_data = dataset.get_stan_data(75)

country:Denmark
First non-zero cases is on day 58, and 30 days before 5 days is day 52
Denmark has 50 of data
1    1.633195e-08
Name: ifr, dtype: float64
country:Italy
First non-zero cases is on day 31, and 30 days before 5 days is day 27
Italy has 75 of data
5    1.244963e-08
Name: ifr, dtype: float64
country:Germany
First non-zero cases is on day 28, and 30 days before 5 days is day 46
Germany has 56 of data
10    1.973191e-08
Name: ifr, dtype: float64
country:Spain
First non-zero cases is on day 32, and 30 days before 5 days is day 40
Spain has 62 of data
6    1.294065e-08
Name: ifr, dtype: float64
country:United_Kingdom
First non-zero cases is on day 31, and 30 days before 5 days is day 43
United_Kingdom has 59 of data
4    1.449061e-08
Name: ifr, dtype: float64
country:France
First non-zero cases is on day 25, and 30 days before 5 days is day 38
France has 64 of data
9    1.255619e-08
Name: ifr, dtype: float64
country:Norway
First non-zero cases is on day 58, and 30 days before 5 

In [30]:
stan_data

{'M': 11,
 'p': 5,
 'x1': array([-1.97350876e-01, -1.92017069e-01, -1.86683261e-01, -1.81349454e-01,
        -1.76015647e-01, -1.70681839e-01, -1.65348032e-01, -1.60014224e-01,
        -1.54680417e-01, -1.49346609e-01, -1.44012802e-01, -1.38678994e-01,
        -1.33345187e-01, -1.28011379e-01, -1.22677572e-01, -1.17343764e-01,
        -1.12009957e-01, -1.06676149e-01, -1.01342342e-01, -9.60085345e-02,
        -9.06747270e-02, -8.53409195e-02, -8.00071121e-02, -7.46733046e-02,
        -6.93394971e-02, -6.40056896e-02, -5.86718822e-02, -5.33380747e-02,
        -4.80042672e-02, -4.26704598e-02, -3.73366523e-02, -3.20028448e-02,
        -2.66690374e-02, -2.13352299e-02, -1.60014224e-02, -1.06676149e-02,
        -5.33380747e-03,  2.64323941e-18,  5.33380747e-03,  1.06676149e-02,
         1.60014224e-02,  2.13352299e-02,  2.66690374e-02,  3.20028448e-02,
         3.73366523e-02,  4.26704598e-02,  4.80042672e-02,  5.33380747e-02,
         5.86718822e-02,  6.40056896e-02,  6.93394971e-02,  7.4

In [66]:
stan_data['pop']

[5792203,
 60461828,
 83783945,
 46754783,
 67886004,
 65273512,
 5421242,
 11589616,
 9006400,
 10099270,
 8654618]

In [67]:
import pystan

In [68]:
sm = pystan.StanModel(file="../../stan-models/base.stan")

INFO:pystan:COMPILING THE C++ CODE FOR MODEL anon_model_26789167272cf46e6188fa9ce826678d NOW.


In [69]:
fit = sm.sampling(data=stan_data,iter=40,warmup=20,chains=2)
print(fit)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)
/home/peter/anaconda3/envs/covid19model/lib/python3.8/site-packages/pystan/misc.py:366: RuntimeWarning: invalid value encountered in sqrt
  sem=msd[:, 1] / np.sqrt(ess), ess=ess, rhat=rhat,


Inference for Stan model: anon_model_26789167272cf46e6188fa9ce826678d.
2 chains, each with iter=40; warmup=20; thin=1; 
post-warmup draws per chain=20, total post-warmup draws=40.

                      mean se_mean      sd     2.5%     25%     50%     75%   97.5%  n_eff   Rhat
mu[1]                 5.63    0.04    0.18     5.07     5.6    5.68    5.69    6.01     16   1.04
mu[2]                 1.66     1.1    1.13     0.53    0.56    1.51    2.66    3.58      1   6.16
mu[3]                 2.39    1.85    1.88     0.53    0.54    2.27    4.17    4.73      1  16.72
mu[4]                 5.16     0.6    0.61     4.21    4.57     5.3    5.75    5.76      1   5.46
mu[5]                  2.5    1.54    1.57     0.93    0.96    2.34    3.96    4.47      1  10.75
mu[6]                 3.97    0.55    0.56     3.39    3.43    3.89    4.51    4.79      1   6.26
mu[7]                 1.96    1.16    1.18      0.8     0.8    1.81    3.08     3.5      1  10.92
mu[8]                 3.01    2.86 

In [ ]:
# save it to the file for later use
import pickle
with open('../results/example.pkl', 'wb') as f:
    pickle.dump(sm, f)

In [70]:
# import arviz as az
fit.extract()

OrderedDict([('mu',
              array([[5.59577613, 2.69556298, 4.43800582, 4.77050461, 3.84366343,
                      4.30525713, 3.40570938, 5.77901898, 4.74471121, 4.54124819,
                      1.72556613],
                     [5.44366333, 2.78746006, 4.05962672, 4.58114827, 3.92414663,
                      4.44670779, 3.00992539, 6.05372972, 4.92300181, 4.52637422,
                      2.00724944],
                     [5.60245574, 2.43672753, 4.22532877, 4.65292556, 4.2973397 ,
                      4.34660673, 3.04489806, 5.84345517, 5.16175212, 4.33759816,
                      2.30404353],
                     [5.5008819 , 2.77992971, 4.30452276, 4.49526131, 3.76467533,
                      4.48319958, 2.81722381, 5.76503503, 5.11468492, 4.51893912,
                      1.74880652],
                     [5.57301258, 2.70254532, 4.0537376 , 4.65745573, 3.95500065,
                      4.71189528, 2.87177969, 5.87054756, 4.89054661, 4.29018372,
                    

In [71]:
dataset.countries

['Denmark',
 'Italy',
 'Germany',
 'Spain',
 'United_Kingdom',
 'France',
 'Norway',
 'Belgium',
 'Austria',
 'Sweden',
 'Switzerland']